# GCP Support - Interactive Tutorial with Mock Data

This notebook demonstrates how to use the GCP Support library to find and export Ground Control Points (GCPs) for drone imagery processing.

## Overview

This tutorial covers:
1. **Parsing H3 cells from manifest files** - Extract H3 cell identifiers from input manifests
2. **Generating mock GCPs** - Create sample GCPs for testing
3. **Finding GCPs using GCPFinder** - Use the main finder class to search for GCPs
4. **Filtering GCPs** - Apply quality filters to GCPs
5. **Exporting GCPs** - Export in formats compatible with MetaShape and ArcGIS Pro

## Prerequisites

- Python 3.8+
- All required packages will be installed in the next cell


In [ ]:
# Install required packages
!pip install -q h3>=3.7.0 requests>=2.31.0 geopandas>=0.14.0 shapely>=2.0.0 pandas>=2.0.0 numpy>=1.24.0

print("✓ Packages installed successfully!")


In [ ]:
# Import necessary libraries
import os
import sys
from datetime import datetime

# Add the parent directory to the path to import gcp_support
# This assumes the notebook is in the gcp_support directory or its parent
notebook_dir = os.getcwd()
script_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# Try multiple paths to find gcp_support
possible_paths = [
    notebook_dir,  # Current directory
    os.path.dirname(notebook_dir),  # Parent directory
    script_dir,  # Script directory
    os.path.dirname(script_dir),  # Parent of script directory
]

for path in possible_paths:
    if os.path.exists(os.path.join(path, 'gcp_support')):
        if path not in sys.path:
            sys.path.insert(0, path)
        break

try:
    from gcp_support import GCPFinder
    from gcp_support.mock_gcp import MockGCPGenerator
    from gcp_support.h3_utils import h3_cells_to_bbox
    from gcp_support.manifest_parser import parse_manifest, get_h3_cells_from_manifest
    print("✓ Imports successful!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\nIf you see this error, make sure:")
    print("1. The gcp_support library is installed: pip install -e .")
    print("2. Or adjust the sys.path in this cell to point to the library location")
    raise


## Step 1: Setup Output Directory

Create a directory to store all generated GCP files.


In [ ]:
# Create output directory
output_dir = './gcps_output'
os.makedirs(output_dir, exist_ok=True)

print(f"✓ Output directory created: {output_dir}")


## Step 2: Parse Manifest File

The manifest file contains information about drone imagery, including H3 cell identifiers. We'll extract the H3 cells to determine the area of interest.


In [ ]:
# Path to the manifest file
# For this notebook, we'll create a sample manifest inline
# In production, you would load this from a file

manifest_content = '''[{"prefix":"s3://spexi-data-domain-assets-production-ca-central-1/standardized-images/89121ab6d23ffff/36247/"},"89121ab6d23ffff_36247_0018.jpg","89121ab6d23ffff_36247_0019.jpg","89121ab6d23ffff_36247_0020.jpg","89121ab6d23ffff_36247_0021.jpg","89121ab6d23ffff_36247_0022.jpg","89121ab6d23ffff_36247_0023.jpg","89121ab6d23ffff_36247_0024.jpg","89121ab6d23ffff_36247_0026.jpg","89121ab6d23ffff_36247_0025.jpg","89121ab6d23ffff_36247_0027.jpg","89121ab6d23ffff_36247_0028.jpg","89121ab6d23ffff_36247_0029.jpg","89121ab6d23ffff_36247_0030.jpg","89121ab6d23ffff_36247_0031.jpg","89121ab6d23ffff_36247_0032.jpg","89121ab6d23ffff_36247_0033.jpg","89121ab6d23ffff_36247_0034.jpg"]'''

# Save manifest to a temporary file
manifest_path = os.path.join(output_dir, 'input-file.manifest')
with open(manifest_path, 'w') as f:
    f.write(manifest_content)

# Parse the manifest
h3_cells, prefix = parse_manifest(manifest_path)

print(f"✓ Parsed manifest: {len(h3_cells)} H3 cell(s) found")
print(f"  H3 cells: {h3_cells}")
if prefix:
    print(f"  S3 prefix: {prefix}")


## Step 3: Convert H3 Cells to Bounding Box

Convert the H3 cell identifiers to a latitude/longitude bounding box that defines our area of interest.


In [ ]:
# Get bounding box from H3 cells
bbox = h3_cells_to_bbox(h3_cells)

print(f"✓ Bounding box calculated:")
print(f"  Min Latitude: {bbox[0]:.6f}")
print(f"  Min Longitude: {bbox[1]:.6f}")
print(f"  Max Latitude: {bbox[2]:.6f}")
print(f"  Max Longitude: {bbox[3]:.6f}")
print(f"\n  Full bbox: {bbox}")


## Step 4: Example 1 - Generate Mock GCPs from Bounding Box

Generate sample GCPs within the bounding box. These mock GCPs simulate what you would get from real GCP sources.


In [ ]:
print("Example 1: Generate mock GCPs from bounding box (from manifest H3 cells)")
print("-" * 70)

# Generate mock GCPs
mock_gcps = MockGCPGenerator.generate_gcps_in_bbox(bbox, count=20)
print(f"✓ Generated {len(mock_gcps)} mock GCPs")

# Display first few GCPs
print("\nSample GCPs:")
for i, gcp in enumerate(mock_gcps[:3]):
    print(f"  GCP {i+1}: {gcp['id']} at ({gcp['lat']:.6f}, {gcp['lon']:.6f}), accuracy: {gcp['accuracy']:.2f}m")


## Step 5: Export GCPs for MetaShape and ArcGIS Pro

Export the GCPs in formats compatible with both MetaShape and ArcGIS Pro.


In [ ]:
# Initialize GCPFinder
finder = GCPFinder()

# Export all formats
finder.export_all(mock_gcps, output_dir, 'mock_bbox')

print(f"✓ Exported all formats to {output_dir}/mock_bbox_*")
print("\nGenerated files:")
files = [f for f in os.listdir(output_dir) if f.startswith('mock_bbox')]
for file in sorted(files):
    filepath = os.path.join(output_dir, file)
    size = os.path.getsize(filepath)
    print(f"  - {file} ({size} bytes)")


## Step 6: Example 2 - Find GCPs Using GCPFinder

Use the GCPFinder class to find GCPs using the H3 cells. This demonstrates the full workflow including USGS/NOAA search and filtering.


In [ ]:
print("Example 2: Find GCPs using GCPFinder with H3 cells from manifest")
print("-" * 70)
print(f"Using H3 cells from manifest: {h3_cells}")

# Find GCPs using the H3 cells
# Note: This uses mock data since we don't have API credentials configured
gcps_from_finder = finder.find_gcps(h3_cells=h3_cells, max_results=20)
print(f"✓ Found {len(gcps_from_finder)} GCPs using GCPFinder")

# Export
finder.export_all(gcps_from_finder, output_dir, 'mock_h3')
print(f"✓ Exported all formats to {output_dir}/mock_h3_*")


## Step 7: Example 3 - Test GCP Filtering

Demonstrate how to filter GCPs based on quality criteria such as accuracy.


In [ ]:
from gcp_support.gcp_filter import GCPFilter

print("Example 3: Test GCP filtering")
print("-" * 70)

# Generate GCPs with varying accuracy
all_gcps = MockGCPGenerator.generate_gcps_in_bbox(
    bbox, 
    count=30, 
    accuracy_range=(0.1, 3.0)  # Accuracy from 0.1m to 3.0m
)
print(f"✓ Generated {len(all_gcps)} GCPs with varying accuracy")

# Filter for high accuracy (<= 1.0m)
filter_obj = GCPFilter(min_accuracy=1.0)
filtered_gcps = filter_obj.filter_gcps(all_gcps)
print(f"✓ Filtered to {len(filtered_gcps)} GCPs with accuracy <= 1.0m")

# Export filtered results
finder.export_all(filtered_gcps, output_dir, 'mock_filtered')
print(f"✓ Exported filtered GCPs to {output_dir}/mock_filtered_*")


## Step 8: Summary - View All Generated Files

Display all the files that were created during this tutorial.


In [ ]:
print("Example 4: Generated files")
print("-" * 70)
files = [f for f in os.listdir(output_dir) if f.startswith('mock_')]
print(f"Total files generated: {len(files)}\n")

for file in sorted(files):
    filepath = os.path.join(output_dir, file)
    size = os.path.getsize(filepath)
    print(f"  {file} ({size} bytes)")

print("\n" + "=" * 70)
print("✓ All examples completed successfully!")
print(f"✓ Check the '{output_dir}' directory for exported files")
print("=" * 70)


## Next Steps

- **For MetaShape**: Import the `.txt` or `.xml` files into Agisoft MetaShape
- **For ArcGIS Pro**: Import the `.csv`, `.geojson`, or `.shp` files into ArcGIS Pro
- **Configure API access**: See `USGS_API_NOTES.md` for instructions on configuring real USGS/NOAA API access
- **Customize filtering**: Adjust the `min_accuracy` and other filter parameters based on your needs

## File Formats

- **MetaShape**: `.txt` (CSV format) and `.xml` (marker file format)
- **ArcGIS Pro**: `.csv` (simple CSV), `.geojson` (GeoJSON format), `.shp` (shapefile with supporting files)
